#Import Lib

In [11]:
!pip install -U "tensorflow>=2.10.0"

In [12]:
!pip install remotezip tqdm opencv-python==4.9.0.80 opencv-python-headless==4.9.0.80 tf-models-official

In [13]:
!pip install tensorflow_hub

In [14]:
!pip install remotezip tqdm opencv-python==4.9.0.80 opencv-python-headless==4.9.0.80 tf-models-official

In [15]:
!pip install remotezip tqdm opencv-python
!pip install -q git+https://github.com/tensorflow/docs

  Preparing metadata (setup.py) ... done


In [16]:
!pip install remotezip tqdm opencv-python einops


In [17]:

from official.projects.movinet.modeling import movinet
from official.projects.movinet.modeling import movinet_model

In [18]:
import tqdm
import random
import pathlib
import itertools
import collections
import keras
import os
import cv2
import numpy as np
import tensorflow
import remotezip as rz

import tensorflow as tf
import keras
from keras import layers
import tensorflow as tf
import einops
import matplotlib.pyplot as plt
import seaborn as sns
# Some modules to display an animation using imageio.
import imageio
from IPython import display
from urllib import request
from tensorflow_docs.vis import embed

# Load data

In [19]:
# #@title

# def list_files_per_class(zip_url):
#   """
#     List the files in each class of the dataset given the zip URL.

#     Args:
#       zip_url: URL from which the files can be unzipped.

#     Return:
#       files: List of files in each of the classes.
#   """
#   files = []
#   with rz.RemoteZip(URL) as zip:
#     for zip_info in zip.infolist():
#       files.append(zip_info.filename)
#   return files

# def get_class(fname):
#   """
#     Retrieve the name of the class given a filename.

#     Args:
#       fname: Name of the file in the UCF101 dataset.

#     Return:
#       Class that the file belongs to.
#   """
#   return fname.split('_')[-3]

# def get_files_per_class(files):
#   """
#     Retrieve the files that belong to each class.

#     Args:
#       files: List of files in the dataset.

#     Return:
#       Dictionary of class names (key) and files (values).
#   """
#   files_for_class = collections.defaultdict(list)
#   for fname in files:
#     class_name = get_class(fname)
#     files_for_class[class_name].append(fname)
#   return files_for_class

# def download_from_zip(zip_url, to_dir, file_names):
#   """
#     Download the contents of the zip file from the zip URL.

#     Args:
#       zip_url: Zip URL containing data.
#       to_dir: Directory to download data to.
#       file_names: Names of files to download.
#   """
#   with rz.RemoteZip(zip_url) as zip:
#     for fn in tqdm.tqdm(file_names):
#       class_name = get_class(fn)
#       zip.extract(fn, str(to_dir / class_name))
#       unzipped_file = to_dir / class_name / fn

#       fn = pathlib.Path(fn).parts[-1]
#       output_file = to_dir / class_name / fn
#       unzipped_file.rename(output_file,)

# def split_class_lists(files_for_class, count):
#   """
#     Returns the list of files belonging to a subset of data as well as the remainder of
#     files that need to be downloaded.

#     Args:
#       files_for_class: Files belonging to a particular class of data.
#       count: Number of files to download.

#     Return:
#       split_files: Files belonging to the subset of data.
#       remainder: Dictionary of the remainder of files that need to be downloaded.
#   """
#   split_files = []
#   remainder = {}
#   for cls in files_for_class:
#     split_files.extend(files_for_class[cls][:count])
#     remainder[cls] = files_for_class[cls][count:]
#   return split_files, remainder

# def download_ufc_101_subset(zip_url, num_classes, splits, download_dir):
#   """
#     Download a subset of the UFC101 dataset and split them into various parts, such as
#     training, validation, and test.

#     Args:
#       zip_url: Zip URL containing data.
#       num_classes: Number of labels.
#       splits: Dictionary specifying the training, validation, test, etc. (key) division of data
#               (value is number of files per split).
#       download_dir: Directory to download data to.

#     Return:
#       dir: Posix path of the resulting directories containing the splits of data.
#   """
#   files = list_files_per_class(zip_url)
#   for f in files:
#     tokens = f.split('/')
#     if len(tokens) <= 2:
#       files.remove(f) # Remove that item from the list if it does not have a filename

#   files_for_class = get_files_per_class(files)

#   classes = list(files_for_class.keys())[:num_classes]

#   for cls in classes:
#     new_files_for_class = files_for_class[cls]
#     random.shuffle(new_files_for_class)
#     files_for_class[cls] = new_files_for_class

#   # Only use the number of classes you want in the dictionary
#   files_for_class = {x: files_for_class[x] for x in list(files_for_class)[:num_classes]}

#   dirs = {}
#   for split_name, split_count in splits.items():
#     print(split_name, ":")
#     split_dir = download_dir / split_name
#     split_files, files_for_class = split_class_lists(files_for_class, split_count)
#     download_from_zip(zip_url, split_dir, split_files)
#     dirs[split_name] = split_dir

#   return dirs

def format_frames(frame, output_size):
  """
    Pad and resize an image from a video.

    Args:
      frame: Image that needs to resized and padded.
      output_size: Pixel size of the output frame image.

    Return:
      Formatted frame with padding of specified output size.
  """
  frame = tf.image.convert_image_dtype(frame, tf.float32)
  frame = tf.image.resize_with_pad(frame, *output_size)
  return frame

def frames_from_video_file(video_path, n_frames, output_size = (224,224), frame_step = 15):
  """
    Creates frames from each video file present for each category.

    Args:
      video_path: File path to the video.
      n_frames: Number of frames to be created per video file.
      output_size: Pixel size of the output frame image.

    Return:
      An NumPy array of frames in the shape of (n_frames, height, width, channels).
  """
  # Read each video frame by frame
  result = []
  src = cv2.VideoCapture(str(video_path))

  video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)

  need_length = 1 + (n_frames - 1) * frame_step

  if need_length > video_length:
    start = 0
  else:
    max_start = video_length - need_length
    start = random.randint(0, max_start + 1)

  src.set(cv2.CAP_PROP_POS_FRAMES, start)
  # ret is a boolean indicating whether read was successful, frame is the image itself
  ret, frame = src.read()
  result.append(format_frames(frame, output_size))

  for _ in range(n_frames - 1):
    for _ in range(frame_step):
      ret, frame = src.read()
    if ret:
      frame = format_frames(frame, output_size)
      result.append(frame)
    else:
      result.append(np.zeros_like(result[0]))
  src.release()
  result = np.array(result)[..., [2, 1, 0]]

  return result

class FrameGenerator:
  def __init__(self, path, n_frames, training = False):
    """ Returns a set of frames with their associated label.

      Args:
        path: Video file paths.
        n_frames: Number of frames.
        training: Boolean to determine if training dataset is being created.
    """
    self.path = path
    self.n_frames = n_frames
    self.training = training
    self.class_names = sorted(set(p.name for p in self.path.iterdir() if p.is_dir()))
    self.class_ids_for_name = dict((name, idx) for idx, name in enumerate(self.class_names))

  def get_files_and_class_names(self):
    video_paths = list(self.path.glob('*/*.mp4'))
    classes = [p.parent.name for p in video_paths]
    return video_paths, classes

  def __call__(self):
    video_paths, classes = self.get_files_and_class_names()

    pairs = list(zip(video_paths, classes))

    if self.training:
      random.shuffle(pairs)

    for path, name in pairs:
      video_frames = frames_from_video_file(path, self.n_frames)
      label = self.class_ids_for_name[name] # Encode labels
      yield video_frames, label

In [20]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
n_frames = 8
batch_size = 8

output_signature = (tf.TensorSpec(shape = (None, None, None, 3), dtype = tf.float32),
                    tf.TensorSpec(shape = (), dtype = tf.int16))

train_ds = tf.data.Dataset.from_generator(FrameGenerator(pathlib.Path("/content/drive/MyDrive/KLTN2023/DataKLTN2023/data_video/train"), n_frames, training=True),
                                          output_signature = output_signature)


#Batch the data
train_ds = train_ds.batch(batch_size)

val_ds = tf.data.Dataset.from_generator(FrameGenerator(pathlib.Path("/content/drive/MyDrive/KLTN2023/DataKLTN2023/data_video/val"), n_frames),
                                         output_signature = output_signature)
val_ds = val_ds.batch(batch_size)

test_ds = tf.data.Dataset.from_generator(FrameGenerator(pathlib.Path("/content/drive/MyDrive/KLTN2023/DataKLTN2023/data_video/test_final"), n_frames),
                                          output_signature = output_signature)

test_ds = test_ds.batch(batch_size)

In [ ]:
test_ds = tf.data.Dataset.from_generator(FrameGenerator(pathlib.Path("/content/drive/MyDrive/KLTN2023/DataKLTN2023/data_video/test_final"), n_frames),
                                          output_signature = output_signature)

test_ds = test_ds.batch(batch_size)
actual = [labels for _, labels in test_ds.unbatch()]


In [ ]:
numpy_array = np.array([tensor.numpy() for tensor in actual])
print(numpy_array)

In [ ]:
for frames, labels in train_ds.take(5):
  print(f"Shape: {frames.shape}")
  print(f"Label: {labels.shape}")

# Model

In [ ]:
gru = layers.GRU(units=4, return_sequences=True, return_state=True)

inputs = tf.random.normal(shape=[1, 10, 8]) # (batch, sequence, channels)

result, state = gru(inputs) # Run it all at once

In [ ]:
print(inputs.shape)

In [ ]:
# first_half, state = gru(inputs[:, :5, :])   # run the first half, and capture the state
# second_half, _ = gru(inputs[:,5:,:], initial_state=state)  # Use the state to continue where you left off.

# print(np.allclose(result[:, :5,:], first_half))
# print(np.allclose(result[:, 5:,:], second_half))

In [ ]:
model_id = 'a0'
resolution = 224

tf.keras.backend.clear_session()

backbone = movinet.Movinet(model_id=model_id)
backbone.trainable = False

# Set num_classes=600 to load the pre-trained weights from the original model
model = movinet_model.MovinetClassifier(backbone=backbone, num_classes=600)
model.build([None, None, None, None, 3])

# Load pre-trained weights
!wget https://storage.googleapis.com/tf_model_garden/vision/movinet/movinet_a0_base.tar.gz -O movinet_a0_base.tar.gz -q
!tar -xvf movinet_a0_base.tar.gz

checkpoint_dir = f'movinet_{model_id}_base'
checkpoint_path = tf.train.latest_checkpoint(checkpoint_dir)
checkpoint = tf.train.Checkpoint(model=model)
status = checkpoint.restore(checkpoint_path)
status.assert_existing_objects_matched()

In [ ]:
def build_classifier(batch_size, num_frames, resolution, backbone, num_classes):
  """Builds a classifier on top of a backbone model."""
  model = movinet_model.MovinetClassifier(
      backbone=backbone,
      num_classes=num_classes)
  model.build([batch_size, num_frames, resolution, resolution, 3])

  return model

In [ ]:
model = build_classifier(batch_size, n_frames, resolution, backbone, 5)

In [ ]:
num_epochs = 5

optimizer = tensorflow.keras.optimizers.Adamax(
    learning_rate=0.0001)
from keras import optimizers
model.compile(loss = tensorflow.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              # optimizer = optimizer,
              metrics = ['accuracy'])

In [ ]:
results = model.fit(train_ds,
                    validation_data=test_ds,
                    epochs=5,
                    validation_freq=1,
                    verbose=1)

In [ ]:
# prompt: save nodel

model.save("/content/drive/MyDrive/KLTN_O_H_2024/movinet.keras")


In [ ]:
def plot_history(history):
  """
    Plotting training and validation learning curves.

    Args:
      history: model history with all the metric measures
  """
  fig, (ax1, ax2) = plt.subplots(2)

  fig.set_size_inches(18.5, 10.5)

  # Plot loss
  ax1.set_title('Loss')
  ax1.plot(history.history['loss'], label = 'train')
  ax1.plot(history.history['val_loss'], label = 'test')
  ax1.set_ylabel('Loss')

  # Determine upper bound of y-axis
  max_loss = max(history.history['loss'] + history.history['val_loss'])

  ax1.set_ylim([0, np.ceil(max_loss)])
  ax1.set_xlabel('Epoch')
  ax1.legend(['Train', 'Validation'])

  # Plot accuracy
  ax2.set_title('Accuracy')
  ax2.plot(history.history['accuracy'],  label = 'train')
  ax2.plot(history.history['val_accuracy'], label = 'test')
  ax2.set_ylabel('Accuracy')
  ax2.set_ylim([0, 1])
  ax2.set_xlabel('Epoch')
  ax2.legend(['Train', 'Validation'])

  plt.show()

plot_history(results)

In [ ]:
model.evaluate(test_ds, return_dict=True)

In [ ]:
def get_actual_predicted_labels(dataset):
  """
    Create a list of actual ground truth values and the predictions from the model.

    Args:
      dataset: An iterable data structure, such as a TensorFlow Dataset, with features and labels.

    Return:
      Ground truth and predicted values for a particular dataset.
  """
  actual = [labels for _, labels in dataset.unbatch()]
  predicted = model.predict(dataset)

  actual = tf.stack(actual, axis=0)
  predicted = tf.concat(predicted, axis=0)
  #predicted = tf.argmax(predicted, axis=1)

  return actual, predicted

In [ ]:
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
def plot_confusion_matrix(actual, predicted, labels, ds_type):
  cm = tf.math.confusion_matrix(actual, predicted)
  ax = sns.heatmap(cm, annot=True, fmt='g')
  sns.set(rc={'figure.figsize':(12, 12)})
  sns.set(font_scale=1.4)
  ax.set_title('Confusion matrix of action recognition for ' + ds_type)
  ax.set_xlabel('Predicted Action')
  ax.set_ylabel('Actual Action')
  plt.xticks(rotation=90)
  plt.yticks(rotation=0)
  ax.xaxis.set_ticklabels(labels)
  ax.yaxis.set_ticklabels(labels)
  plt.show()
plot_confusion_matrix(actual, predicted, labels, 'test')

In [ ]:
fg = FrameGenerator(pathlib.Path("/content/drive/MyDrive/KLTN2023/DataKLTN2023/data_video/test_final"), n_frames, training=True)
labels = list(fg.class_ids_for_name.keys())
actual, predicted = get_actual_predicted_labels(test_ds)
predicted

In [ ]:
confusion_matrix = np.array([[9, 1, 0, 0, 0],
                             [0, 16, 3, 2, 1],
                             [0, 1, 12, 0, 0],
                             [2, 1, 0, 4, 0],
                             [3, 3, 2, 1, 17]])

In [ ]:
predicted

In [ ]:
labels

In [ ]:
def calculate_classification_metrics(y_actual, y_pred, labels):
  """
    Calculate the precision and recall of a classification model using the ground truth and
    predicted values.

    Args:
      y_actual: Ground truth labels.
      y_pred: Predicted labels.
      labels: List of classification labels.

    Return:
      Precision and recall measures.
  """
  cm = tf.math.confusion_matrix(y_actual, y_pred)
  tp = np.diag(cm) # Diagonal represents true positives
  precision = dict()
  recall = dict()
  for i in range(len(labels)):
    col = cm[:, i]
    fp = np.sum(col) - tp[i] # Sum of column minus true positive is false negative

    row = cm[i, :]
    fn = np.sum(row) - tp[i] # Sum of row minus true positive, is false negative

    precision[labels[i]] = tp[i] / (tp[i] + fp) # Precision

    recall[labels[i]] = tp[i] / (tp[i] + fn) # Recall

  return precision, recall

precision, recall = calculate_classification_metrics(actual, predicted, labels) # Test dataset

In [ ]:
precision, recall = calculate_classification_metrics(actual, predicted, labels) # Test dataset
# Calculate Macro Precision
macro_precision = sum(precision.values()) / len(precision)

# Calculate Macro Recall
macro_recall = sum(recall.values()) / len(recall)

# Calculate Macro F1-score
macro_f1_score = (2 * macro_precision * macro_recall) / (macro_precision + macro_recall)

print("Macro Precision:", macro_precision)
print("Macro Recall:", macro_recall)
print("Macro F1-score:", macro_f1_score)

In [ ]:
print(precision)
print(recall)

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

def extract_frames(video_path, output_path):
    video_capture = cv2.VideoCapture(video_path)
    frame_count = 0
    fps = int(video_capture.get(cv2.CAP_PROP_FPS))
    frame_interval = fps // 2  # Extract every 2 frames per second

    while True:
        ret, frame = video_capture.read()
        if not ret:
            break

        frame_count += 1
        if frame_count % frame_interval == 0:
            cv2.imwrite(output_path.format(frame_count), frame)
            cv2_imshow(frame)  # Display image using cv2_imshow() in Colab
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    video_capture.release()

# Example usage:
video_path = '/content/drive/MyDrive/KLTN2023/DataKLTN2023/data_video/train/violent/0004.mp4'
output_path = '/content/drive/MyDrive/KLTN2023/DataKLTN2023/data_video/frames/violent_{}.jpg'
extract_frames(video_path, output_path)
